# Calculate Area of wetlands

***
## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load Packages

In [1]:
import datacube
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rioxarray as rio
import xarray as xr
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog
from datacube.utils.masking import mask_invalid_data
from deafrica_tools.classification import sklearn_flatten, sklearn_unflatten
from deafrica_tools.plotting import display_map, plot_lulc
from matplotlib import colors as mcolors
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Analysis Parameters

* `pred_tif`: The path and name of the prediction GeoTIFF output in the previous notebook.
* `ouput_dir`: A folder location to store the classified GeoTIFFs.

In [2]:
year = "2017"
output_dir = "results"

pred_tif = f"{output_dir}/prediction_{year}.tif"

# Coordinates for the classified area from 4_Classify_satellite_data.ipynb
new_lat, new_lon = 1.3733, 32.2903 #centred on Uganda
buf_lat, buf_lon = 1, 1


class_dict = {
    "Cropland": 1,
    "Dense Forest": 2,
    "Open Forest": 3,
    "Open Grassland": 4,
    "Open Water": 5,
    "Otherland": 6,
    "Settlements": 7,
    "Vegetated Wetland": 8,
    "Wooded Grassland": 9,
}

In [3]:
# Load the  pixel-based prediction raster.
pred = rio.open_rasterio(pred_tif).squeeze()
pred

<xarray.DataArray (y: 12754, x: 9649)>
[123063346 values with dtype=float64]
Coordinates:
    band         int64 1
  * x            (x) float64 3.019e+06 3.019e+06 ... 3.212e+06 3.212e+06
  * y            (y) float64 3.027e+05 3.027e+05 ... 4.765e+04 4.763e+04
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [4]:
# Get the resolution i.e. actual ground distance represented by the length of a single pixel in meters.
pixel_length = pred.geobox.resolution[1]
# Conversion from metres to kilometres.
m_per_km = 1000
# Get the actual ground area represented by each pixel.
area_per_pixel = (pixel_length / m_per_km) ** 2

In [5]:
# Use the numpy np.unique function to return the pixel count for each land cover class.
counts = np.unique(pred, return_counts=True)
# Calculate the area of each land cover class.
areas = np.array(counts[1] * area_per_pixel)
results = dict(zip(class_dict.keys(), areas))
# Convert the results dictionary into a pandas DataFrame.
reults_df = pd.DataFrame.from_dict(results, orient='index', columns=[f"Area in {year} ($km^2$)"])
reults_df

,Area in 2017 ($km^2$)
Cropland,26932.6932
Dense Forest,2740.6432
Open Forest,3836.8288
Open Grassland,4942.7872
Open Water,3475.5656
Otherland,12.1496
Settlements,125.2700
Vegetated Wetland,1989.1968
Wooded Grassland,5170.2040
